In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#Since I use Colab, I want to export pickle to my Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
sparse_matrix_pickle_path = '/content/drive/MyDrive/AnimeRecommenderPickle/UserAnimeList_Pivot.pkl' #After training, you can delete this
feature_sparse_mat_path = '/content/drive/MyDrive/AnimeRecommenderPickle/UserAnime_Pivot.pkl'
anime_db_parquet_path = '/content/drive/MyDrive/AnimeRecommenderPickle/anime_db.parquet' #Anime details
trained_model_path = '/content/drive/MyDrive/AnimeRecommenderPickle/trained_model.pkl'
anime_index_path = '/content/drive/MyDrive/AnimeRecommenderPickle/anime_index.pkl' #Needed for inference and convert return features to id

In [ ]:

import scipy, pickle
from scipy.sparse import csr_matrix
import pandas as pd
import numpy as np
from pandas.arrays import SparseArray
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors

with open(sparse_matrix_pickle_path, 'rb') as f:
  sparse_matrix = pickle.load(f)

%time sparse_matrix = sparse_matrix.fillna(0)

CPU times: user 48.4 s, sys: 6.03 s, total: 54.5 s
Wall time: 55 s


In [ ]:
import gc
gc.collect()
%time csr_mat = csr_matrix(sparse_matrix.sparse.to_dense())

In [ ]:
#Train the model
#For sparse matrix only brute force is available
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
model_knn.fit(csr_mat)

In [ ]:
with open('UserAnime_Pivot.pkl', 'wb') as f:
  pickle.dump(csr_mat, f)

!cp "UserAnime_Pivot.pkl" "/content/drive/MyDrive/AnimeRecommenderPickle/"

In [ ]:
with open('anime_index.pkl', 'wb') as f:
  pickle.dump(sparse_matrix.index, f)

!cp "anime_index.pkl" "/content/drive/MyDrive/AnimeRecommenderPickle/"

In [ ]:
gc.collect()
with open(trained_model_path, 'wb') as f:
  pickle.dump(model_knn, f)

In [ ]:
with open(anime_index_path, 'rb') as f:
  anime_index = pickle.load(f)

with open(feature_sparse_mat_path, 'rb') as f:
  features_mat = pickle.load(f)

with open(trained_model_path, 'rb') as f:
  trained_model = pickle.load(f)

def recommend_anime(mal_id: str):
  mal_id = int(mal_id)

  matrix_index = anime_index.get_loc(mal_id)

  anime_features = features_mat[matrix_index, :]
  #Inference
  neighbors = model_knn.kneighbors(anime_features, return_distance=False)

  #Convert back to MAL ID
  mal_ids = anime_index[neighbors.ravel()]

  mal_ids = [i for i in mal_ids if i != mal_id]
  return mal_ids

recommend_anime(51179)

In [ ]:
for item in anime_db[anime_db['anime_id'] == 2366]['studios']:
  print(item)

In [ ]:
anime_db = pd.read_parquet(anime_db_parquet_path)
def get_anime(mal_id):
  mal_id = int(mal_id)
  return anime_db[anime_db['anime_id'] == mal_id].iloc[0, : ]

def get_animes(mal_ids):
  return anime_db.set_index('anime_id').loc[mal_ids, :]

def search_anime(query_str):
  query_str = query_str.lower()
  query_df = anime_db[anime_db['title'].str.lower().str.contains(query_str)]
  return query_df.sort_values('popularity_rank', ascending=True)

get_anime(40221)